# Question-4

Imagine you working as a sale manager now you need to predict the Revenue
and whether that particular revenue is on the weekend or not and find the
Informational_Duration using the Ensemble learning algorithm.

Data_link - https://www.kaggle.com/datasets/henrysue/online-shoppers-intention

In [1]:
# importing necessary libraries
import pandas as pd       # for data analysis
import numpy as np          # for numerical calculation
import seaborn as sns               # for visualisation
import matplotlib.pyplot as plt   # for visualisation
%matplotlib inline

# importing ml algorithms

from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder,StandardScaler,OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.metrics import accuracy_score,classification_report
from sklearn.model_selection import train_test_split,GridSearchCV
import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv("online_shoppers_intention.csv")   # read the dataset 

In [3]:
data   # print data

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0,0.0,0,0.0,1,0.000000,0.200000,0.200000,0.000000,0.0,Feb,1,1,1,1,Returning_Visitor,False,False
1,0,0.0,0,0.0,2,64.000000,0.000000,0.100000,0.000000,0.0,Feb,2,2,1,2,Returning_Visitor,False,False
2,0,0.0,0,0.0,1,0.000000,0.200000,0.200000,0.000000,0.0,Feb,4,1,9,3,Returning_Visitor,False,False
3,0,0.0,0,0.0,2,2.666667,0.050000,0.140000,0.000000,0.0,Feb,3,2,2,4,Returning_Visitor,False,False
4,0,0.0,0,0.0,10,627.500000,0.020000,0.050000,0.000000,0.0,Feb,3,3,1,4,Returning_Visitor,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12325,3,145.0,0,0.0,53,1783.791667,0.007143,0.029031,12.241717,0.0,Dec,4,6,1,1,Returning_Visitor,True,False
12326,0,0.0,0,0.0,5,465.750000,0.000000,0.021333,0.000000,0.0,Nov,3,2,1,8,Returning_Visitor,True,False
12327,0,0.0,0,0.0,6,184.250000,0.083333,0.086667,0.000000,0.0,Nov,3,2,1,13,Returning_Visitor,True,False
12328,4,75.0,0,0.0,15,346.000000,0.000000,0.021053,0.000000,0.0,Nov,2,2,3,11,Returning_Visitor,False,False


In [4]:
data.shape   # give rows and columns

(12330, 18)

In [5]:
data.isnull().sum()   # Checking the null values

Administrative             0
Administrative_Duration    0
Informational              0
Informational_Duration     0
ProductRelated             0
ProductRelated_Duration    0
BounceRates                0
ExitRates                  0
PageValues                 0
SpecialDay                 0
Month                      0
OperatingSystems           0
Browser                    0
Region                     0
TrafficType                0
VisitorType                0
Weekend                    0
Revenue                    0
dtype: int64

In [6]:
data.duplicated().sum()      #  # Checking the  duplicated data 

125

In [7]:
data.drop_duplicates(inplace=True)           # removing  the  duplicated data 

In [8]:
data.info()   # gives information of data

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12205 entries, 0 to 12329
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Administrative           12205 non-null  int64  
 1   Administrative_Duration  12205 non-null  float64
 2   Informational            12205 non-null  int64  
 3   Informational_Duration   12205 non-null  float64
 4   ProductRelated           12205 non-null  int64  
 5   ProductRelated_Duration  12205 non-null  float64
 6   BounceRates              12205 non-null  float64
 7   ExitRates                12205 non-null  float64
 8   PageValues               12205 non-null  float64
 9   SpecialDay               12205 non-null  float64
 10  Month                    12205 non-null  object 
 11  OperatingSystems         12205 non-null  int64  
 12  Browser                  12205 non-null  int64  
 13  Region                   12205 non-null  int64  
 14  TrafficType           

In [9]:
data["Revenue"] = data["Revenue"].map({True:1,False:0})  # mappping / convert booling to numerical
data['Weekend'] = data['Weekend'].map({True:1,False:0})  # mappping / convert booling to numerical

In [11]:
# Convert informational duration column to numerical values (0 for False, 1 for True)
data['Informational_Duration'] = data['Informational_Duration'].apply(lambda x: 1 if x > 0 else 0)

In [12]:
catigorical_features = data.select_dtypes(include="object").columns   # separate numwerical and catigorical features
numerical_features = data.select_dtypes(exclude="object").columns
print(catigorical_features)
print(numerical_features)

Index(['Month', 'VisitorType'], dtype='object')
Index(['Administrative', 'Administrative_Duration', 'Informational',
       'Informational_Duration', 'ProductRelated', 'ProductRelated_Duration',
       'BounceRates', 'ExitRates', 'PageValues', 'SpecialDay',
       'OperatingSystems', 'Browser', 'Region', 'TrafficType', 'Weekend',
       'Revenue'],
      dtype='object')


In [13]:
from sklearn.preprocessing import LabelEncoder    # use label encoding on catigorical data
lable = LabelEncoder()

for i in catigorical_features:
    data[i] = lable.fit_transform(data[i])

In [21]:
X = data.drop(labels=["Revenue"],axis=1)    ## Independent and dependent features
Y = data[["Revenue"]]

In [29]:
catigorical_features = X.select_dtypes(include="object").columns     # separate numerical and catigorical features
numerical_features = X.select_dtypes(exclude="object").columns
print(catigorical_features)
print(numerical_features)

Index([], dtype='object')
Index(['Administrative', 'Administrative_Duration', 'Informational',
       'Informational_Duration', 'ProductRelated', 'ProductRelated_Duration',
       'BounceRates', 'ExitRates', 'PageValues', 'SpecialDay', 'Month',
       'OperatingSystems', 'Browser', 'Region', 'TrafficType', 'VisitorType',
       'Weekend'],
      dtype='object')


In [30]:
X  

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend
0,0,0.0,0,0,1,0.000000,0.200000,0.200000,0.000000,0.0,2,1,1,1,1,2,0
1,0,0.0,0,0,2,64.000000,0.000000,0.100000,0.000000,0.0,2,2,2,1,2,2,0
2,0,0.0,0,0,1,0.000000,0.200000,0.200000,0.000000,0.0,2,4,1,9,3,2,0
3,0,0.0,0,0,2,2.666667,0.050000,0.140000,0.000000,0.0,2,3,2,2,4,2,0
4,0,0.0,0,0,10,627.500000,0.020000,0.050000,0.000000,0.0,2,3,3,1,4,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12325,3,145.0,0,0,53,1783.791667,0.007143,0.029031,12.241717,0.0,1,4,6,1,1,2,1
12326,0,0.0,0,0,5,465.750000,0.000000,0.021333,0.000000,0.0,7,3,2,1,8,2,1
12327,0,0.0,0,0,6,184.250000,0.083333,0.086667,0.000000,0.0,7,3,2,1,13,2,1
12328,4,75.0,0,0,15,346.000000,0.000000,0.021053,0.000000,0.0,7,2,2,3,11,2,0


In [31]:
Y

,Revenue
0,0
1,0
2,0
3,0
4,0
...,...
12325,0
12326,0
12327,0
12328,0


In [32]:
## Numerical Pipline
num_pipline = Pipeline(
    steps=[
        ("imputer",SimpleImputer(strategy="median")),
        ("scaler",StandardScaler())
    ]
)

## categorical Pipline
cato_pipline = Pipeline(
    steps=[
        ("imputer",SimpleImputer(strategy="most_frequent")),
        ("scaler",StandardScaler())
    ]
)

# Create Preprocessor object
preprocessor = ColumnTransformer([
    ("num_pipline",num_pipline,numerical_features),
    ("cato_pipline",cato_pipline,catigorical_features)
])

In [33]:
from sklearn.model_selection import train_test_split      # split train - test data
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.20,random_state=42)

In [34]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())  # use fit-tranform fun
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out()) # use tranform fun

In [36]:
models={'RandomForest': RandomForestClassifier(),
       'GradientBoost': GradientBoostingClassifier()}   # use required models

params={'RandomForest':{'n_estimators': [100,300,500],'max_depth':[15],'min_samples_split':[5],'random_state':[1]},
       'GradientBoost':{'n_estimators': [100,300,500],'max_depth':[15],'min_samples_split' :[5],'random_state':[1]}}
           # use hyper-parameter tunning

In [37]:
def evaluate_models(X_train,X_test,y_train,y_test,models,params):     # function to get best model
    report={}
    for i in range(len(list(models))):
        model=list(models.values())[i]
        param=params[list(models.keys())[i]]
        
        gs=GridSearchCV(model,param,cv=3)
        gs.fit(X_train,y_train)
        
        model.set_params(**gs.best_params_)
        model.fit(X_train,y_train)
        
        y_test_pred=model.predict(X_test)
        acc_score=accuracy_score(y_test,y_test_pred)
        
        print(f'{list(models.keys())[i]} :')
        print(classification_report(y_test,y_test_pred))
        
        report[list(models.keys())[i]]=acc_score
        
        best_score=max(list(report.values()))
        best_model=list(filter(lambda x:report[x]==best_score,report))[0]
    print(f'The Best score of models:{best_model} : {best_score}')
    return report

In [38]:
evaluate_models(X_train,X_test,y_train,y_test,models,params)   # call the function

RandomForest :
              precision    recall  f1-score   support

           0       0.93      0.97      0.95      2079
           1       0.75      0.55      0.64       362

    accuracy                           0.91      2441
   macro avg       0.84      0.76      0.79      2441
weighted avg       0.90      0.91      0.90      2441

GradientBoost :
              precision    recall  f1-score   support

           0       0.93      0.96      0.94      2079
           1       0.71      0.56      0.63       362

    accuracy                           0.90      2441
   macro avg       0.82      0.76      0.78      2441
weighted avg       0.89      0.90      0.90      2441

The Best score of models:RandomForest : 0.9065956575174109


{'RandomForest': 0.9065956575174109, 'GradientBoost': 0.9008603031544449}

In [39]:
best_mod=models['RandomForest']
gs=GridSearchCV(best_mod,params['RandomForest'],cv=3)  # apply hyper-parameter tunning on randon forest
gs.fit(X_train,y_train)

GridSearchCV(cv=3,
             estimator=RandomForestClassifier(max_depth=15, min_samples_split=5,
                                              n_estimators=500,
                                              random_state=1),
             param_grid={'max_depth': [15], 'min_samples_split': [5],
                         'n_estimators': [100, 300, 500], 'random_state': [1]})

In [40]:
best_mod.set_params(**gs.best_params_)    # apply hyper-parameter tunning on randon forest
best_mod.fit(X_train,y_train)

RandomForestClassifier(max_depth=15, min_samples_split=5, n_estimators=500,
                       random_state=1)

In [59]:
y_pred=best_mod.predict(X_test)     # predict label

In [61]:
revenue_new=pd.DataFrame(y_pred)    # make dataframe for predicted data
revenue_new.columns=['revenue']
X_set=pd.DataFrame(X_test)  # make dataframe for X_test data

In [62]:
df_new=pd.concat([X_set,revenue_new],axis='columns')  # concatenate two dataframes

In [63]:
df_new.shape  # got shape of data - rows and columns

(2441, 18)

In [64]:
df_new.head()  # 1st five rows of new dataframe

,num_pipline__Administrative,num_pipline__Administrative_Duration,num_pipline__Informational,num_pipline__Informational_Duration,num_pipline__ProductRelated,num_pipline__ProductRelated_Duration,num_pipline__BounceRates,num_pipline__ExitRates,num_pipline__PageValues,num_pipline__SpecialDay,num_pipline__Month,num_pipline__OperatingSystems,num_pipline__Browser,num_pipline__Region,num_pipline__TrafficType,num_pipline__VisitorType,num_pipline__Weekend,revenue
0,-0.701239,-0.463971,-0.399102,-0.499872,-0.293049,-0.034587,-0.450980,-0.745201,-0.320068,-0.306329,-2.182654,-0.141539,0.967127,-0.061923,-0.514443,0.409328,-0.551161,0
1,0.198761,1.084374,-0.399102,-0.499872,-0.471997,-0.439427,-0.450980,-0.730568,0.616344,-0.306329,1.191343,-0.141539,-0.210381,-0.895295,-0.514443,0.409328,-0.551161,1
2,-0.701239,-0.463971,-0.399102,-0.499872,-0.650944,-0.575031,-0.450980,1.030226,-0.320068,-0.306329,-0.073906,-1.252463,-0.210381,-0.895295,0.983477,0.409328,-0.551161,0
3,-0.701239,-0.463971,-0.399102,-0.499872,-0.114101,-0.307710,-0.095634,-0.463669,-0.320068,-0.306329,0.769594,-0.141539,-0.210381,-0.061923,-0.764097,0.409328,1.814351,0
4,-0.101239,-0.169048,0.374233,2.000512,-0.315417,0.031648,-0.450980,-0.690851,-0.320068,-0.306329,-0.073906,-0.141539,-0.210381,-0.895295,-0.514443,-2.476363,-0.551161,0
